In [1]:
import os
import struct
import torch
import numpy as np
from tqdm import tqdm

def read_aedat_file(filename):
    """Read a single .aedat file and return parsed data"""
    with open(filename, 'rb') as f:
        header_lines = []
        while True:
            pos = f.tell()
            line = f.readline()
            if not line:
                raise ValueError(f"No binary data section found in file {filename}, please check file format")

            try:
                decoded_line = line.decode('ascii', errors='strict')
            except UnicodeDecodeError:
                # Unable to decode ASCII, means this is the start of binary data
                f.seek(pos)
                break

            stripped_line = decoded_line.strip()
            if stripped_line.startswith('#'):
                header_lines.append(stripped_line)
            else:
                f.seek(pos)
                break

        data_start_index = f.tell()  # Data section start offset
        data = f.read()

    event_size = 8
    num_events = len(data) // event_size

    addresses = []
    timestamps = []
    xs = []
    ys = []
    polarities = []

    for i in range(num_events):
        event_data = data[i * event_size:(i + 1) * event_size]
        # Parse address and timestamp in big-endian order
        address, t = struct.unpack('>ii', event_data)
        polarity = address & 1
        x = (address >> 1) & 0x7F
        y = (address >> 8) & 0x7F

        addresses.append(address)
        timestamps.append(t)
        xs.append(x)
        ys.append(y)
        polarities.append(polarity)

    return {
        'header': header_lines,
        'addresses': addresses,
        'timestamps': timestamps,
        'xs': xs,
        'ys': ys,
        'polarities': polarities
    }

def process_and_save_histogram(input_base_folder, output_base_folder, img_size=(128, 128)):
    """Process all .aedat files in category folders and save histogram_two_channel"""
    # Iterate through category folders
    for class_folder in os.listdir(input_base_folder):
        input_folder = os.path.join(input_base_folder, class_folder)
        output_folder = os.path.join(output_base_folder, class_folder)

        if not os.path.isdir(input_folder):
            continue  # Skip non-folder items

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)  # Create corresponding output folder

        for filename in tqdm(os.listdir(input_folder), desc=f"Processing {class_folder}"):
            if not filename.endswith('.aedat'):
                continue

            filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, filename.replace('.aedat', '.pt'))

            # Read and parse file
            data = read_aedat_file(filepath)
            xs, ys, polarities = data['xs'], data['ys'], data['polarities']
            num_events = len(xs)

            # Initialize two channels, one for positive polarity events, one for negative polarity events
            positive_channel = np.zeros(img_size, int)
            negative_channel = np.zeros(img_size, int)

            for i in range(num_events):
                x, y, polarity = xs[i], ys[i], polarities[i]
                if polarity > 0:
                    positive_channel[img_size[0] - 1 - y, x] += 1
                else:
                    negative_channel[img_size[0] - 1 - y, x] += 1

            # Stack positive and negative channels as 2x128x128
            histogram_two_channel = np.stack([positive_channel, negative_channel])

            # Convert to PyTorch format and save
            torch_histogram = torch.tensor(histogram_two_channel, dtype=torch.float32)
            torch.save(torch_histogram, output_filepath)

        print(f"Category '{class_folder}' processed and saved to {output_folder}")

# Set input and output paths
input_base_folder = r"C:\\Users\\Lem17\\Master Thesis\\Data processing\\data_aedat2"
output_base_folder = "./bipolar_stack"

# Execute processing
process_and_save_histogram(input_base_folder, output_base_folder)

Processing .ipynb_checkpoints: 0it [00:00, ?it/s]


Category '.ipynb_checkpoints' processed and saved to ./bipolar_stack\.ipynb_checkpoints


Processing airplane: 100%|█████████████████████████████████████████████████████████| 1000/1000 [02:56<00:00,  5.67it/s]


Category 'airplane' processed and saved to ./bipolar_stack\airplane


Processing automobile: 100%|███████████████████████████████████████████████████████| 1000/1000 [03:53<00:00,  4.27it/s]


...

Category 'horse' processed and saved to ./bipolar_stack\horse


Processing ship: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [02:50<00:00,  5.87it/s]


Category 'ship' processed and saved to ./bipolar_stack\ship


Processing truck: 100%|████████████████████████████████████████████████████████████| 1000/1000 [03:38<00:00,  4.59it/s]

Category 'truck' processed and saved to ./bipolar_stack\truck
